<a href="https://colab.research.google.com/github/matech96/DeepLearningAVizualisInformatikaban/blob/master/HF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Deep Learning Alkalmazása a Vizuális Informatikában
##2. Házi Feladat

###1. Rész

Valósíts meg egy paraméterezhető konvolúciós neurális hálózatot, amely osztályozásra képes. A hálózat paraméterei a következők:


*   nC: Az osztályok száma
*   nFeat: Az első réteg kimeneti csatornaszáma. Az ezt követő rétegek be- és kimeneti csatornaszáma egyezzen ezzel meg, majd minden leskálázó (strided konvolúciós) réteg duplázza ezt meg.
*   nLevels: A háló szintjeinek száma. Egy szintnek az azonos térbeli kiterjedésű tenzorokon operáló rétegeket nevezzük (2 leskálázás közt). (Tipp: használj adaptív poolingot az osztályozó réteg előtt, hogy a változó szint szám ne okozzon problémát.)
*   layersPerLevel: Az egy szinten található konvolúciós rétegek száma.
*   kernelSize: A konvolúciós rétegek mérete.
*   nLinType: Kategorikus változó, amellyel a nemlinearitás típusát állíthatja (hogy hány és milyen függvények közül lehet választani önre van bízva)
*   bNorm: Bináris változó, amellyel állítható, hogy a konvolúciós rétegekbe teszünk-e BatchNormot.
*   dropOut: Az osztályozó réteget közvetlenül megelőző dropout réteg bemeneti valószínűsége
*   residual: Bináris változó, True érték esetén minden szinten valósíts meg egy reziduális kapcsolatot a szint bemenete és a szint végén megjelenő leskálázó réteg bemenete közt.

Tipp: Érdemes ehhez írni először két külön modult, ami egy réteget (batchnormmal, dropouttal és nemlinearitással) valósít meg és egyet, ami meg ezekből egy szintet legózik össze. Ezekből aránylag könnyen összelegózható a háló.

Tipp 2: A PyTorchnak van nn.ModuleList osztálya. Ez egy lista, amiben rétegek vannak, de ha sima lista változóba tesztek egyszerre több nn.Module-t, annak az optimizer nem fogja megkapni. Ez azért van, mert amikor egy nn.Module-tól leszármazó osztálytól elkéritek a .parameters()-t, akkor az végignézi az objektum összes tagváltozót, hogy van-e neki .parameters() függvénye (és ezt szépen rekurzívan végigcsinálja). A Listának pedig nincs, hiába vannak benne olyan elemek, amiknek van. +1: van nn.Sequential is, ami ugyanaz, csak a forward függvénye is felül van csapva, és szépen sorban meghívja a belül lévő rétegeket.



In [1]:
!pip install pytorch-lightning
!pip install bayesian-optimization

In [0]:
import torch as th
from torch import nn

class Conv(nn.Sequential):
  def __init__(self, in_ch, out_ch, kernel_size, act_name, is_bnorm, stride=1):
    super().__init__()
    act_fn = str_to_act_fn(act_name)
    self.add_module(f"conv", nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=kernel_size, padding=kernel_size//2, stride=stride))
    self.add_module(f"{act_name}", act_fn())
    if is_bnorm:
      self.add_module(f"bnorm", nn.BatchNorm2d(out_ch))

class Level(nn.Sequential):
  def __init__(self, ch, n_conv, kernel_size, act_name, is_bnorm):
    super().__init__()
    for conv_id in range(n_conv-1):
      self.add_module(f"conv_{conv_id}", Conv(ch, ch, kernel_size, act_name, is_bnorm))
    # self.add_module(f"conv_{conv_id}", Conv(in_ch, out_ch, kernel_size, act_name, is_bnorm, stride=2))

def str_to_act_fn(act_name):
  return nn.modules.activation.__dict__[act_name]
def get_available_act_names():
  return [name for name in nn.modules.activation.__dict__.keys() if name[0].isupper() and name != 'F']

In [0]:
class NN(nn.Module):
  def __init__(self, n_classes, color_ch, img_side, first_out_ch, n_conv, kernel_size, act_name, is_bnorm, is_residual, n_levels):
    super().__init__()
    self.is_residual = is_residual
    self.first = Conv(in_ch=color_ch, out_ch=first_out_ch, kernel_size=kernel_size, act_name=act_name, is_bnorm=False)
    self.convs = nn.ModuleList([Level(ch=first_out_ch*(2**layer_id), 
                                      # out_ch=first_out_ch*(2**(layer_id+1)), 
                                      n_conv=n_conv, 
                                      kernel_size=kernel_size,
                                      act_name=act_name,
                                      is_bnorm=is_bnorm) for layer_id in range(n_levels)])
    self.stride_convs = nn.ModuleList([Conv(in_ch=first_out_ch*(2**layer_id), 
                                            out_ch=first_out_ch*(2**(layer_id+1)), 
                                            kernel_size=kernel_size,
                                            act_name=act_name,
                                            is_bnorm=is_bnorm,
                                            stride=2) for layer_id in range(n_levels)])
    # self.pools = nn.ModuleList([nn.MaxPool2d(kernel_size=2) for _ in range(n_levels)])
    lst_side = img_side // (2**n_levels)
    lst_ch = first_out_ch * (2**(n_levels))
    self.fc = nn.Linear(lst_side*lst_side*lst_ch, n_classes)
    
  def forward(self, x):
    x_out = self.first(x)
    for conv, stride_conv in zip(self.convs, self.stride_convs):
    # for conv in self.convs:
      x_in = x_out

      x_out = conv(x_in)
      if self.is_residual:
        x_out = th.add(x_in, x_out)      
      x_out = stride_conv(x_out)

    x_out = th.flatten(x_out, start_dim=1)
    return self.fc(x_out)

In [0]:
# from torchsummary import summary
# summary(NN(n_classes=10, color_ch=3, img_side=32, first_out_ch=8, n_conv=2, kernel_size=3, act_name='ReLU', is_bnorm=True, is_residual=True, n_levels=3), (3,32,32))

###2. Rész

Valósíts meg egy paraméterezhető neurális háló tanító függvényt. 

A függvény bemenetként megkapja a fenti neurális háló megkostruálásához szükséges paramétereket, az összes random seedet 42 értékre állítja, majd végrehajtja a neurális háló tanítását a CIFAR10 adatbázison.

Használj Adam optimizert és Cosine Annealing tanulási ráta ütemezőt.

A tanítás során minden epoch után validálj, és jegyezd fel a legjobb validációs pontosságot, és a tanítás végén ezt add vissza.

A függvénynek további bemeneti paraméterei:

*   bSize: A bacth méret
*   lr: a tanulási ráta
*   lr_ratio: a tanulási ráta ütemező eta_min paramétere és a kezdeti tanulási ráta hányadosa
*   numEpoch: az epochok száma
*   decay: a weight_decay paraméter értéke



In [0]:
import random
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR


class CoolCIFAR(pl.LightningModule):  
  def __init__(self, first_out_ch, n_conv, kernel_size, act_name, is_bnorm, is_residual, n_levels, bSize, lr, lr_ratio, decay, numEpoch):
    super().__init__()
    self.bSize = bSize
    self.lr = lr
    self.lr_ratio = lr_ratio
    self.decay = decay
    self.numEpoch = numEpoch
    self.model = NN(10, 3, 32, first_out_ch, n_conv, kernel_size, act_name, is_bnorm, is_residual, n_levels)
    self.loss = nn.CrossEntropyLoss()

    self.transform = transforms.Compose(
                    [transforms.ToTensor(),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  def parameters(self):
    return self.model.parameters()

  def forward(self, x):
    return self.model(x)

  def train_dataloader(self):
    return DataLoader(CIFAR10('/cifar',download=True, train=True, transform=self.transform), batch_size=self.bSize)

  def val_dataloader(self):
    return DataLoader(CIFAR10('/cifar',download=True, train=False, transform=self.transform), batch_size=self.bSize)

  def test_dataloader(self):    
    return self.val_dataloader()

  def configure_optimizers(self):
    opt = Adam(self.parameters(), lr=self.lr, weight_decay=self.decay)
    sch = CosineAnnealingLR(opt, self.numEpoch, eta_min=self.lr_ratio)
    return [opt], [sch]

  def training_step(self, batch, batch_idx):
    x, y = batch
    res = self.forward(x)
    loss = self.loss(res, y)
    logs = {'train_loss': loss}
    return {'loss': loss, 'log': logs}

  def test_step(self, batch, batch_idx):
    return self.validation_step(batch, batch_idx)

  def validation_step(self, batch, batch_idx):
    x, y = batch
    res = self.forward(x)    
    loss = self.loss(res, y)

    y_ = th.argmax(res, dim=1)
    acc = th.sum(y == y_).item() / (len(y) * 1.0)
    return {'val_loss': loss, 'val_acc': acc}

  def test_epoch_end(self, outputs):
    self.test_res = self.validation_end(outputs)
    return self.test_res

  def validation_end(self, outputs):
    avg_loss = th.stack([x['val_loss'] for x in outputs]).mean()
    avg_acc = np.stack([x['val_acc'] for x in outputs]).mean()
    # tensorboard_logs = {'val_loss': avg_loss}
    res = {'avg_val_loss': avg_loss, 'avg_val_acc': avg_acc}
    # print(res)
    return res

def train(first_out_ch, n_conv, kernel_size, act_name, is_bnorm, is_residual, n_levels, bSize, lr, lr_ratio, decay, numEpoch):
  random.seed(42)
  np.random.seed(42)
  th.manual_seed(42)
  th.backends.cudnn.deterministic = True
  th.backends.cudnn.benchmark = False

  model = CoolCIFAR(first_out_ch, n_conv, kernel_size, act_name, is_bnorm, is_residual, n_levels, bSize, lr, lr_ratio, decay, numEpoch)
  # print(summary(model.model, (3, 32, 32)))
  trainer = Trainer(min_epochs=1, max_epoch=numEpoch, show_progress_bar=False)#, train_percent_check=0.01, val_percent_check=0.01, fast_dev_run=True)
  trainer.fit(model)
  trainer.test()
  return trainer.model.test_res['avg_val_acc']


In [0]:
# train(first_out_ch=8, n_conv=2, kernel_size=3, act_name='ReLU', is_bnorm=True, is_residual=True, n_levels=3, bSize=64, lr=0.1, lr_ratio=0.0, decay=0.01, numEpoch=3)

###3. Rész

Valósíts meg hiperparaméter optimalizálást a Bayesian Optimization python könyvtár felhasználásával. A könyvtár itt érhető el: https://github.com/fmfn/BayesianOptimization

A megoldás során a következőkre ügyelj:


1.   A kezdeti random lépések száma legyen kb egyenlő a szabad paraméterek számának felével (5-6)
2.   Az teljes lépésszám legyen ennek tízszerese (50-60)
3.   Mivel a Bayesian Optimization függvény a bináris/diszkrét/integer paramétereket nem támogatja, ezért a folytonos értékek megfelelő konverziója az előző feladatrészben megvalósított függvény feladata.
4.   Vannak persze olyan paraméterek, amiket nem kell optimalizálni (pl.: BatchNorm ami True, vagy az nClass ami 10 CIFAR10 esetében)
5.   Az egyes paraméterek tartományát nektek kell ésszerűen meghatározni.
6.   Az epochok közben menő progress barokat meg kiíratásokat érdemes eltüntetni, a Bayesian opt majd fog írogatni
7.   Mivel ez sokáig tart érdemes ellenőrizni kevés adaton (van külön erre Sampler, amit a DataLoader-nek lehet átadni), hogy sikerül-e az overfitting.
8.   Referenciaként, a CIFAR10-en olyan 91-2% pontosság az elfogadható, 95% számít jónak és 97%+ a state-of-the-art. Ezt persze a kis adatokon nem fogjátok elérni, mert az overfittinges.



In [0]:
from bayes_opt import BayesianOptimization
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

def f(first_out_ch, n_conv, kernel_size, n_levels, lr, lr_ratio, decay):
  blockPrint()
  val_acc = train(first_out_ch=int(first_out_ch),
               n_conv=int(n_conv),
               kernel_size=int(kernel_size)*2 + 1,
               act_name='ReLU',
               is_bnorm=True,
               is_residual=True, 
               n_levels=int(n_levels),
               bSize=64, 
               lr=lr, 
               lr_ratio=lr_ratio,
               decay=decay,
               numEpoch=50)
  enablePrint()
  return val_acc

pbounds = {'first_out_ch': (2, 8),
           'n_conv': (1, 3),
           'kernel_size': (0, 2),
           'n_levels': (1, 3),
           'lr': (0.0001, 0.1),
           'lr_ratio': (0.0001, 0.1),
           'decay': (0.0001, 0.1)
           }

optimizer = BayesianOptimization(
    f=f,
    pbounds=pbounds,
)

optimizer.maximize(
    init_points=5,
    n_iter=50,
)